In [12]:
import numpy as np
import pandas as pd
import unidecode
import os

In [236]:
pd.set_option('display.max_rows',999)
pd.set_option('display.min_rows',10)
pd.set_option('display.float_format', '{:,.2f}'.format)

In [19]:
path = r'C:/Users/Franco/Dropbox/files/properati/'

In [16]:
mylist = []

for file in os.listdir(path):
    if file.endswith(".csv"):
        mylist += [file]

In [23]:
prope = pd.DataFrame()

for file in mylist:
    df = pd.read_csv(path+'{}'.format(file))
    prope = prope.append(df)

In [25]:
prope = prope.drop(columns=['ad_type','rooms','bedrooms','bathrooms','title','description','lat','lon'])
prope = prope[prope['l1'].isin(['Argentina', 'Uruguay','Perú', 'Colombia','Ecuador'])]
prope = prope.loc[prope['operation_type']=='Venta',:]
prope = prope[prope['property_type'].isin(['Casa','Departamento','PH','Apartamento'])]

In [26]:
[prope['Country'],prope['City']] = [prope['l1'],prope['l3']]

prope = prope[prope['price'].notna()]
prope = prope[prope['City'].notna()]

Cambio de Buenos Aires y Montevideo

In [27]:
prope = prope.set_index(['l2'])

In [28]:
prope.at['Capital Federal','City'] = 'Ciudad de Buenos Aires'

In [29]:
prope.at['Montevideo','City'] = 'Montevideo'

In [30]:
prope = prope.reset_index()

In [31]:
prope = prope.replace({'Country':{'Perú':'Peru'}})

CAMBIAR NOBMRES

In [201]:
#Prope precios de prop's comerciales para obtener nombres limpios
p = pd.read_excel(r'C:\Users\Franco\Dropbox\files\clean\prope\comercial\properati_i.xlsx',index_col=[0,1])

In [202]:
p = p.loc[:,'City Original'].reset_index().drop_duplicates().rename(columns={'City':'City cw','City Original':'City'}).set_index(['Country','City'])

In [203]:
p = p.reset_index().append(pd.DataFrame([['Argentina','San Miguel de Tucumán','Gran Tucuman-Tafi Viejo']],columns=['Country','City','City cw'])).set_index(['Country','City'])

In [209]:
prope['City'] = prope['City'].apply(lambda x: unidecode.unidecode(x))

In [210]:
prope['Country'] = prope['Country'].apply(lambda x: unidecode.unidecode(x))

In [211]:
prope = prope.set_index(['Country','City'])

In [212]:
prope = prope.join(p,how='inner')

In [213]:
prope['fecha'] = prope['created_on'].str.strip()

In [214]:
prope = prope[prope['currency'].isin(['USD','COP'])]

In [215]:
prope = prope.loc[:,['surface_total','surface_covered','price','currency','property_type','City cw','fecha']]

Calcular metros promedio por pais y tipo de propiedad

In [219]:
inds = prope[['surface_total','surface_covered']].isnull().all(axis=1)

In [220]:
me = prope.loc[~inds,:]

In [237]:
def r(row):
    if row['property_type'] in ['Departamento','Apartamento']:
        val = 'Departamento'
    else:
        val = 'Casa'
    return val

In [221]:
me['Tipo Propiedad'] = me.apply(r,axis=1)

C:\ProgramData\Anaconda3\envs\caf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [222]:
me = me.groupby(['Country','Tipo Propiedad'])['surface_total','surface_covered'].mean()

C:\ProgramData\Anaconda3\envs\caf\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [225]:
me.to_csv(r'C:/Users/Franco/Dropbox\files/clean/prope/vivienda/metros_promedio.csv')

Currency

In [36]:
serie = os.listdir(r'c:/Users/Franco/Dropbox/files/properati')[0]

In [37]:
tc = pd.read_excel(r'c:/Users/Franco/Dropbox/files/properati/{}'.format(serie),usecols=[0,1],skiprows=[0,1,2,3,4,5,6]).dropna()

In [38]:
tc['fecha'] = tc[tc.columns[0]].astype(str).str.split(' ').str[0].str.strip()

In [39]:
tc = tc.rename(columns={'Tasa de cambio representativa del mercado (TRM)':'TDC'})

Join

In [226]:
prope = prope.drop(columns=['surface_total','surface_covered']) 

In [227]:
prope['price'] = prope['price'].astype('int')

Parto con las df's de prope porq es demasiado grande

In [228]:
prope = prope.reset_index().set_index('Country')

In [229]:
prope_no = prope.drop('Colombia')

In [230]:
prope_col = prope.reset_index()[prope.reset_index()['Country']=='Colombia']

In [231]:
prope_col = prope_col.merge(tc.loc[:,['fecha','TDC']],on='fecha',how='left',indicator=True,validate='m:1')

In [232]:
prope_no['TDC'] = np.nan

C:\ProgramData\Anaconda3\envs\caf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [233]:
prope_no = prope_no.reset_index()

In [234]:
prope = prope_no.append(prope_col.drop(columns='_merge'))

In [235]:
def f(row):
    if row['currency'] == 'USD':
        val = row['price']
    else:
        val = row['price'] / row['TDC']
    return val

In [236]:
prope['Precio USD'] = prope.apply(f,axis=1)

In [237]:
def r(row):
    if row['property_type'] in ['Departamento','Apartamento']:
        val = 'Departamento'
    else:
        val = 'Casa'
    return val

In [238]:
prope['Tipo Propiedad'] = prope.apply(r,axis=1)

In [239]:
prope = prope.rename(columns={'City':'City Original','City cw':'City'}).set_index(['Country','City'])

In [240]:
prope_vivienda = prope.drop(columns=['price','TDC','property_type']).groupby(['Country','City','Tipo Propiedad']).mean()

In [244]:
prope.to_csv(r'C:/Users/Franco/Dropbox\files/clean/prope/vivienda/prope_vivienda.csv')

In [245]:
prope_vivienda.to_csv(r'C:/Users/Franco/Dropbox\files/clean/prope/vivienda/prope_vivienda_promedios.csv')

### Ver tema cantidad de salarios para comprar vivienda

In [246]:
cw = pd.read_excel(r'C:/Users/Franco/Dropbox/files/clean/cwpops.xlsx',index_col=[0,1])

In [247]:
laboral = pd.read_excel(r'C:/Users/Franco/Dropbox/files/clean/laboral_2.xlsx',index_col=[0,1])

In [248]:
salarios = cw.loc[:,'pops'].to_frame().join(laboral.loc[:,'Salario Horario (PPP 2011)'],how='inner')

In [249]:
salarios['Salario Mensual'] = salarios['Salario Horario (PPP 2011)'] * 175

In [250]:
prope_vivienda = prope_vivienda.unstack()

In [259]:
salarios = salarios.join(prope_vivienda,how='inner').rename(columns={('Precio USD', 'Casa'):'Precio Promedio Casa (USD)',('Precio USD', 'Departamento'):'Precio Promedio Departamento (USD)'})

In [261]:
[salarios['Cantidad de salarios mensuales para adquirir Casa'],salarios['Cantidad de salarios mensuales para adquirir Departamento']] = [salarios['Precio Promedio Casa (USD)']/salarios['Salario Mensual'],salarios['Precio Promedio Departamento (USD)']/salarios['Salario Mensual']]

In [263]:
salarios.to_csv(r'C:/Users/Franco/Dropbox\files/clean/prope/vivienda/salariosxviviendo.csv')

### Envío Guille 2021-02-23

In [112]:
path = r'C:/Users/Franco/Dropbox/files/properati/'

In [132]:
mylist = []

for file in os.listdir(path):
    if file.endswith(".csv"):
        mylist += [file]

In [133]:
prope = pd.DataFrame()

for file in mylist:
    df = pd.read_csv(path+'{}'.format(file))
    prope = prope.append(df)

In [134]:
prope = prope[prope['l1'].isin(['Argentina', 'Uruguay','Perú', 'Colombia','Ecuador'])]
prope = prope.loc[prope['operation_type']=='Venta',:]
prope = prope[prope['property_type'].isin(['Casa','Departamento','PH','Apartamento'])]

In [135]:
[prope['Country'],prope['City']] = [prope['l1'],prope['l3']]

prope = prope[prope['price'].notna()]
prope = prope[prope['City'].notna()]

Cambio de Buenos Aires y Montevideo

In [136]:
prope = prope.set_index(['l2'])
prope.at['Capital Federal','City'] = 'Ciudad de Buenos Aires'
prope.at['Montevideo','City'] = 'Montevideo'
prope = prope.reset_index()
prope = prope.replace({'Country':{'Perú':'Peru'}})

Otros ajustes

In [137]:
prope = prope.drop(columns=['ad_type','l1','l2','l3','l4','l5','l6','operation_type'])

In [138]:
prope = prope[prope.columns.tolist()[-2:] + [prope.columns.tolist()[-3]] + prope.columns.tolist()[:-3]]

In [139]:
prope = prope.drop(columns=['id','title','description','price_period'])

In [140]:
prope = prope[(prope['price']!=0)]

In [141]:
prope = prope[~((prope['surface_total'].isna())&(prope['surface_covered']).isna())]

In [142]:
prope = prope[((prope['surface_total']!=0)|(prope['surface_covered'])!=0)]

In [143]:
prope = prope.set_index(['Country','City'])

In [144]:
prope['fecha'] = prope['created_on'].str.strip()

#### Tipo de cambio

In [145]:
tc = pd.read_excel(path+'1.1.1.TCM_Serie histórica IQY.xlsx',usecols=[0,1],skiprows=[0,1,2,3,4,5,6]).dropna()
tc['fecha'] = tc[tc.columns[0]].astype(str).str.split(' ').str[0].str.strip()
tc = tc.rename(columns={'Tasa de cambio representativa del mercado (TRM)':'TDC'})

Parto con las df's de prope porq es demasiado grande

In [146]:
prope2 = prope.reset_index().set_index('Country')
prope_no = prope2.drop('Colombia')
prope_no['TDC'] = np.nan
prope_no = prope_no[prope_no['currency']=='USD']
prope_no = prope_no.reset_index()

prope_col = prope2.reset_index()[prope.reset_index()['Country']=='Colombia']
prope_col = prope_col.merge(tc.loc[:,['fecha','TDC']],on='fecha',how='left',indicator=True,validate='m:1')

prope2 = prope_no.append(prope_col.drop(columns='_merge'))

C:\ProgramData\Anaconda3\envs\caf\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [147]:
def f(row):
    if row['currency'] == 'USD':
        val = row['price']
    else:
        val = row['price'] / row['TDC']
    return val

In [148]:
prope2['Precio USD'] = prope2.apply(f,axis=1)

In [167]:
prope2 = prope2[prope2['property_type']!='PH']

In [168]:
def r(row):
    if row['property_type'] in ['Departamento','Apartamento']:
        val = 'Departamento'
    else:
        val = 'Casa'
    return val

In [173]:
prope2['Tipo Propiedad'] = prope2.apply(r,axis=1)

In [189]:
def q05(x):
    return x.quantile(.05)

def q95(x):
    return x.quantile(.95)

In [195]:
med = med.droplevel(0,axis=1)

In [197]:
prope2 = prope2.set_index(['Country','Tipo Propiedad']).join(med.loc[:,['q05','q95']])

In [207]:
prope2 = prope2[(prope2['surface_covered']>prope2['q05'])&(prope2['surface_covered']<prope2['q95'])]

In [210]:
prope2 = prope2.reset_index()

In [232]:
prope2.loc[:,['Country','Tipo Propiedad','surface_covered']].groupby(['Country','Tipo Propiedad']).agg({'surface_covered':['min',q05,q95, 'max','mean','median','count']})

surface_covered                                      \
                                     min   q05    q95      max   mean median   
Country   Tipo Propiedad                                                       
Argentina Casa                     68.00 80.00 358.00   439.00 189.98 178.00   
          Departamento             30.00 32.00 120.00   156.00  61.79  54.00   
Colombia  Casa                     70.00 80.00 450.00   558.00 216.48 188.00   
          Departamento             47.00 53.00 195.00   246.00  98.82  86.00   
Ecuador   Casa                     89.00 96.60 474.40   650.00 240.73 190.00   
          Departamento             58.00 64.00 271.35   319.00 131.67 113.00   
Peru      Casa                     74.00 90.00 623.50 1,101.00 245.68 180.00   
          Departamento             55.00 62.00 211.00   254.00 113.89 103.00   
Uruguay   Casa                     61.00 80.00 700.00   998.00 282.44 224.00   
          Departamento             15.00 33.00 181.00   270.00  81.81  69.00   

                                  
                           count  
Country   Tipo Propiedad          
Argentina Casa             68343  
          Departamento    122732  
Colombia  Casa             16856  
          Departamento     37435  
Ecuador   Casa               153  
          Departamento       114  
Peru      Casa              3876  
          Departamento      8802  
Uruguay   Casa              2456  
          Departamento      5922

In [231]:
prope2.loc[:,['Country','Tipo Propiedad','Precio USD']].groupby(['Country','Tipo Propiedad']).agg({'Precio USD':['min',q05,q95, 'max','mean','median','count']})

Precio USD                                       \
                                min       q05          q95           max   
Country   Tipo Propiedad                                                   
Argentina Casa             5,000.00 70,000.00   710,000.00 26,500,000.00   
          Departamento     6,009.00 55,000.00   415,000.00 33,918,902.00   
Colombia  Casa             5,050.49 40,763.41   578,383.63 42,466,193.88   
          Departamento     5,692.42 38,761.70   329,421.21 38,942,620.66   
Ecuador   Casa            30,000.00 66,000.00   690,000.00  1,650,000.00   
          Departamento    59,130.00 77,600.00   373,750.00    795,000.00   
Peru      Casa             5,000.00 70,000.00   990,000.00 13,500,000.00   
          Departamento     5,000.00 60,000.00   410,135.00 17,122,206.00   
Uruguay   Casa            23,000.00 85,000.00 1,292,500.00 11,000,000.00   
          Departamento     5,000.00 81,525.00   620,000.00  3,500,000.00   

                                                        
                               mean     median   count  
Country   Tipo Propiedad                                
Argentina Casa           303,319.62 240,000.00   68343  
          Departamento   180,879.73 133,500.00  122732  
Colombia  Casa           205,324.88 137,136.59   16856  
          Departamento   131,995.28  99,374.54   37435  
Ecuador   Casa           261,812.19 180,000.00     153  
          Departamento   169,046.19 137,000.00     114  
Peru      Casa           367,076.76 250,000.00    3876  
          Departamento   187,081.31 145,000.00    8802  
Uruguay   Casa           463,623.13 315,000.00    2456  
          Departamento   238,462.59 170,634.00    5922

In [214]:
prope2['Precio x m2'] = prope2['Precio USD'] / prope2['surface_covered']

In [233]:
prope2.loc[:,['Country','Tipo Propiedad','Precio x m2']].groupby(['Country','Tipo Propiedad']).agg({'Precio x m2':['min',q05,q95, 'max','mean','median','count']})

Precio x m2                                        \
                                 min      q05      q95        max     mean   
Country   Tipo Propiedad                                                     
Argentina Casa                 16.67   541.18 2,774.19 139,473.68 1,557.58   
          Departamento         65.79 1,200.72 4,666.67 754,284.47 2,867.50   
Colombia  Casa                 32.39   307.55 1,789.95 589,808.25   925.84   
          Departamento         63.05   583.73 2,226.74 725,575.81 1,285.05   
Ecuador   Casa                105.88   486.21 1,766.44   5,108.36 1,050.44   
          Departamento        507.61   779.98 1,894.37   2,623.76 1,303.73   
Peru      Casa                 26.09   420.33 2,938.42 168,750.00 1,566.74   
          Departamento         36.76   678.86 2,780.26 260,755.89 1,641.52   
Uruguay   Casa                 74.00   304.56 3,867.55  41,111.11 1,809.86   
          Departamento         68.24 1,341.11 4,708.84  34,090.91 2,907.96   

                                           
                           median   count  
Country   Tipo Propiedad                   
Argentina Casa           1,409.09   68343  
          Departamento   2,544.97  122732  
Colombia  Casa             786.47   16856  
          Departamento   1,140.12   37435  
Ecuador   Casa             927.27     153  
          Departamento   1,315.13     114  
Peru      Casa           1,356.36    3876  
          Departamento   1,444.44    8802  
Uruguay   Casa           1,554.09    2456  
          Departamento   2,777.78    5922

In [240]:
prope3 = prope2.groupby(['Country','City','Tipo Propiedad']).median()

In [241]:
prope3['Precio 60m2'] = prope3['Precio x m2'] * 60

In [244]:
prope4 = prope2.groupby(['Country','City']).median()

In [245]:
prope4['Precio 60m2'] = prope4['Precio x m2'] * 60

In [250]:
prope4.groupby(['Country','City']).median().drop(columns=['lat','lon','rooms','bedrooms','bathrooms','TDC','q05','q95'])

surface_total  surface_covered  \
Country   City                                                              
Argentina 9 de Julio                               50.00            42.00   
          Acambuco                                 55.00            55.00   
          Aeroparque                              420.00           100.00   
          Agua de Oro                             394.00           124.00   
          Aguas Verdes                            110.00            94.00   
          Alberti                                 413.00            74.00   
          Aldao                                    95.00           225.00   
          Alderetes                                  nan            90.00   
          Almirante Brown                         200.00           120.50   
          Alta Gracia                             220.00           155.00   
          Aluminé                               1,000.00           120.00   
          Alvear                                  300.00            88.00   
          Anillaco                                200.00           350.00   
          Anisacate                               150.00           120.00   
          Arequito                                180.00           170.00   
          Arrecifes                               378.00           181.00   
          Arroyo Leyes                            800.00           123.00   
          Arroyo Seco                             394.00            63.50   
          Arroyo de Los Patos                   1,350.00            91.00   
          Ascochinga                            5,000.00           230.00   
          Avellaneda                               70.00            64.00   
          Ayacucho                                254.00           130.50   
          Azul                                    174.00           123.50   
          Añelo                                    55.00            56.00   
          Bahía Blanca                             75.00            70.00   
          Balcarce                                 87.00            62.50   
          Balnearia                             3,409.00           350.00   
          Balneario Orense                        469.00            80.00   
          Baradero                                420.00           190.00   
          Beazley                                  12.00            34.00   
          Benito Juárez                           215.00           215.00   
          Berazategui                             198.00           143.00   
          Berisso                                 180.00           109.00   
          Bialet Massé                            535.00           116.00   
          Bolívar                                    nan           120.00   
          Bouwer                                     nan           200.00   
          Bragado                                  73.00            63.00   
          Brandsen                                300.00           170.00   
          Brazo Largo                             350.00            80.00   
          Cabalango                             1,317.00           115.00   
          Cacheuta                                   nan           300.00   
          Cafayate                                250.00           150.00   
          Caleta Olivia                           370.00           160.00   
          Camet                                   216.00           182.00   
          Campana                                 700.00           160.00   
          Campo Andino                          1,450.00           330.00   
          Candelaria                              675.00           110.00   
          Capilla del Monte                       700.00            90.00   
          Capilla del Rosario                     116.00            90.00   
          Capitán Bermúdez                        286.00           121.00   
          Capitán Sarmiento            

### Nombres

In [389]:
propef = prope4.reset_index()

In [387]:
cw = pd.read_excel(r'C:\Users\Franco\Dropbox\files\clean\cwpops.xlsx').ffill()

In [388]:
cw['Country'] = cw['Country'].apply(lambda x: unidecode.unidecode(x))
cw['City'] = cw['City'].apply(lambda x: unidecode.unidecode(x))

In [380]:
#Prope precios de prop's comerciales para obtener nombres limpios
p = pd.read_excel(r'C:\Users\Franco\Dropbox\files\clean\prope\comercial\properati_i.xlsx',index_col=[0,1])
p = p.loc[:,'City Original'].reset_index().drop_duplicates().rename(columns={'City':'City cw','City Original':'City'})
#p = p.reset_index().append(pd.DataFrame([['Argentina','San Miguel de Tucumán','Gran Tucuman-Tafi Viejo']],columns=['Country','City','City cw'])).set_index(['Country','City'])

In [382]:
pgba = pd.DataFrame({'Country':['Argentina'] * 24,
'City' : ['Avellaneda',
'General San Martín',
'Hurlingham',
'Ituzaingó',
'José C.Paz',
'Lanús',
'Lomas de Zamora',
'Malvinas Argentinas',
'Morón',
'Quilmes',
'San Isidro',
'Tres de Febrero',
'Vicente López',
'Almirante Brown',
'Berazategui',
'Esteban Echeverría',
'Ezeiza',
'Florencio Varela',
'La Matanza',
'Merlo',
'Moreno',
'San Fernando',
'San Miguel',
'Tigre'],
'City cw':['Partidos del GBA'] * 24})

In [386]:
p = p.append(pgba).append(pd.DataFrame({'Country':['Argentina'],'City':['Ciudad de Buenos Aires'],'City cw':['Ciudad de Buenos Aires']})).append(pd.DataFrame({'Country':['Argentina'],'City':['San Miguel de Tucumán'],'City cw':['Gran Tucuman-Tafi Viejo']})).set_index(['Country','City']).sort_values(['Country','City'])

In [390]:
propef['City'] = propef['City'].apply(lambda x: unidecode.unidecode(x))

In [391]:
propef['Country'] = propef['Country'].apply(lambda x: unidecode.unidecode(x))

In [392]:
propef = propef.set_index(['Country','City'])

In [393]:
prope_cw = propef.join(p,how='inner')

In [397]:
gba = prope_cw[prope_cw['City cw']=='Partidos del GBA']

In [403]:
gba = gba.mean().to_frame().T

In [404]:
[gba['Country'],gba['City'],gba['City cw']] = ['Argentina','Partidos del GBA','Partidos del GBA']

In [405]:
pro = prope_cw[prope_cw['City cw']!='Partidos del GBA']

In [409]:
pro = pro.append(gba.set_index(['Country','City']))

### Sueldos

In [414]:
laboral = pd.read_excel(r'C:/Users/Franco/Dropbox/files/clean/laboral_final(ajustes).xlsx',index_col=[0,1])
cw = pd.read_excel(r'C:/Users/Franco/Dropbox/files/clean/cwpops.xlsx',index_col=[0,1])

In [416]:
salarios = cw.loc[:,'pops'].to_frame().join(laboral.loc[:,'Salario Horario (PPP 2011)'],how='inner')
salarios['Salario Mensual'] = salarios['Salario Horario (PPP 2011)'] * 175

In [427]:
pro = pro.reset_index().rename(columns={'City':'City pro','City cw':'City'}).set_index(['Country','City'])
pro = pro.drop(columns=['lat','lon','bedrooms','bathrooms','TDC','price','q05','q95'])

In [430]:
salarios = salarios.join(pro,how='inner')

In [431]:
salarios['Cantidad de salarios mensuales para adquirir Propiedad 60m2'] = salarios['Precio 60m2']/salarios['Salario Mensual']

In [433]:
salarios.to_csv(r'C:/Users/Franco/Dropbox/files/clean/prope/vivienda/salariosxviviendaII.csv')